In [7]:
import sched, time
import matplotlib.pyplot as plt
import numpy as np
import gsw
import xarray as xr
import pandas as pd
from scipy.io import netcdf
import seaborn as sns
sns.set(style='whitegrid',context='poster')
import cmocean.cm as cmo
import glob
from dask import delayed

In [49]:
@delayed
def read_cdffile(cdffile):
    time = cdffile.split('/')[-1].split('.')[0].split('_')[1]
    path = './output_2013asiri_05b/'
    ds = xr.open_dataset(cdffile,drop_variables=['conv','conv100','tr','consump','p'])
    ds['vor']=ds['vor']/gsw.f(18)
    zgrid = pd.read_csv(path+'zgrid.out', skipinitialspace=True, sep=' ', header=None)[1][1:51]
    ds['z'] = zgrid
    ds['time'] = time
    return ds

In [68]:
@delayed
def plot_single_snapshot_withrim(ds):
    path = './output_2013asiri_05b/'
    
    fig, ((ax1, ax3, ax5,ax7), (ax2, ax4, ax6,ax8)) = plt.subplots(2, 4, figsize=(30, 15))
    levels=[20,21,22,24,26,27]

    v = np.linspace(23, 28.7, 101)
    ax1.contourf(ds.yc, ds.xc, ds.temp[32,:,:].T, v, cmap=cmo.thermal,extend='both')
    CS = ax1.contour(ds.yc,ds.xc, ds.rho[32,:,:].T-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax1.set(title='Temperature %05d' % ds.time, ylabel='x', xticks=[])
    h1 = ax2.contourf(ds.yc, ds.z, ds.temp[:,:,1], v, cmap=cmo.thermal, extend='both')
    CS = ax2.contour(ds.yc, ds.z, ds.rho[:,:,1]-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax2.set(xlabel='y', ylabel='z', ylim=(-300, 0))

    cbar_ax = fig.add_axes([0.295, 0.15, 0.01, 0.7]) #
    cbar = fig.colorbar(h1, cax=cbar_ax,ticks=np.arange(15,30,2))
    cbar.ax.set_xticklabels(np.arange(16,30,2))

    ax1.annotate('TOP',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='white',fontweight='bold')
    ax2.annotate('FACE',(0.02,0.02),xycoords='axes fraction',fontsize=20,color='white',fontweight='bold')

    v = np.linspace(31.55, 35.1, 101)
    ax3.contourf(ds.yc, ds.xc, ds.s[32,:,:].T, v, cmap=cmo.haline,extend='both')
    CS = ax3.contour(ds.yc,ds.xc, ds.rho[32,:,:].T-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax3.set(title='Salinity %05d' % ds.time, yticks=[], xticks=[])
    h2 = ax4.contourf(ds.yc, ds.z, ds.s[:,:,1], v, cmap=cmo.haline,extend='both')
    CS = ax4.contour(ds.yc, ds.z, ds.rho[:,:,1]-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax4.set(xlabel='y', ylim=(-300, 0), yticks=[])

    cbar_ax = fig.add_axes([0.498, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(h2, cax=cbar_ax,ticks=np.arange(30,36,1))
    cbar.ax.set_xticklabels(np.arange(30,36,1))

    v = np.linspace(-4, 4, 101)
    ax5.contourf(ds.yc, ds.xc, ds.vor[32,:,:].T, v, cmap=cmo.curl, extend='both')
    CS = ax5.contour(ds.yc,ds.xc, ds.rho[32,:,:].T-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax5.set(title='Vorticity %05d' % ds.time, yticks=[], xticks=[])
    h3 = ax6.contourf(ds.yc, ds.z, ds.vor[:,:,1], v, cmap=cmo.curl, extend='both')
    CS = ax6.contour(ds.yc, ds.z, ds.rho[:,:,1]-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax6.set(xlabel='y', ylim=(-300, 0), yticks=[])

    cbar_ax = fig.add_axes([0.7, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(h3, cax=cbar_ax,ticks=np.arange(-10,11,2))
    cbar.ax.set_xticklabels(np.arange(-4,5,2))

    v = np.linspace(20.2, 25, 101)
    ax7.contourf(ds.yc, ds.xc, ds.rho[32,:,:].T-1000, v, cmap=cmo.dense, extend='both')
    CS = ax7.contour(ds.yc,ds.xc, ds.rho[32,:,:].T-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax7.set(title='Density %05d' % ds.time, yticks=[], xticks=[])
    h4 = ax8.contourf(ds.yc, ds.z, ds.rho[:,:,1]-1000, v, cmap=cmo.dense, extend='both')
    CS = ax8.contour(ds.yc, ds.z, ds.rho[:,:,1]-1000,levels=levels,cmap='Greys_r',linewidths=1)
    plt.clabel(CS, inline=1, fontsize=15,fmt='%2.0f')
    ax8.set(xlabel='y', ylim=(-300, 0), yticks=[])

    cbar_ax = fig.add_axes([0.902, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(h4, cax=cbar_ax,ticks=np.arange(19,26,2),)
    cbar.ax.set_xticklabels(np.arange(-4,5,2))

    plt.subplots_adjust(wspace=0.2, hspace=0.10)

    filename='update_s_t_vor_%05d.png' % ds.time
    plt.savefig(path+'/figures/'+filename, bbox_inches='tight')
    print('plotted %05d' %ds.time)
    plt.close()
    return

In [69]:
template = './output_2013asiri_05b/full_{:05d}.cdf'
filenames = (template.format(k) for k in range(0,20050,100))
datasets = (read_cdffile(fname) for fname in filenames)
plots = (plot_single_snapshot_withrim(ds)for ds in datasets)
_ = [plot.compute() for plot in plots]

plotted 00000
plotted 00100
plotted 00200
plotted 00300
plotted 00400
plotted 00500
plotted 00600
plotted 00700
plotted 00800
plotted 00900
plotted 01000
plotted 01100
plotted 01200
plotted 01300
plotted 01400
plotted 01500
plotted 01600
plotted 01700
plotted 01800
plotted 01900
plotted 02000
plotted 02100
plotted 02200
plotted 02300
plotted 02400
plotted 02500
plotted 02600
plotted 02700
plotted 02800
plotted 02900
plotted 03000
plotted 03100
plotted 03200
plotted 03300
plotted 03400
plotted 03500
plotted 03600
plotted 03700
plotted 03800
plotted 03900
plotted 04000
plotted 04100
plotted 04200
plotted 04300
plotted 04400
plotted 04500
plotted 04600
plotted 04700
plotted 04800
plotted 04900
plotted 05000
plotted 05100
plotted 05200
plotted 05300
plotted 05400
plotted 05500
plotted 05600
plotted 05700
plotted 05800
plotted 05900
plotted 06000
plotted 06100
plotted 06200
plotted 06300
plotted 06400
plotted 06500
plotted 06600
plotted 06700
plotted 06800
plotted 06900
plotted 07000
plotte